# Tarefa 03

- Leia os enunciados com atenção
- Saiba que pode haver mais de uma resposta correta
- Insira novas células de código sempre que achar necessário
- Em caso de dúvidas, procure os Tutores
- Divirta-se :)

In [2]:
import pandas as pd
import requests
from urllib.parse import quote

####  1) Lendo de APIs
Vimos em aula como carregar dados públicos do governo através de um API (*Application Programming Interface*). No exemplo de aula, baixamos os dados de pedidos de verificação de limites (PVL) realizados por estados, e selecionamos apenas aqueles referentes ao estado de São Paulo.

1. Repita os mesmos passos feitos em aula, mas selecione os PVLs realizados por municípios no estado do Rio de Janeiro.
2. Quais são os três *status* das solicitações mais frequentes na base? Quais são suas frequências?
3. Construa uma nova variável que contenha o ano do **status**. Observe que ```data_status``` vem como tipo *object* no **DataFrame**. Dica: você pode usar o método ```.str``` para transformar o tipo da variável em string, em seguida um método como [**slice()**](https://pandas.pydata.org/docs/reference/api/pandas.Series.str.slice.html) ou [**split()**](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.split.html).
4. Indique a frequência de cada ano do campo construído no item (3).

* https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl?uf=RJ&tipo_interessado=Estado

In [11]:
# 1) Seu código aqui
url = 'https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl?uf=RJ&tipo_interessado=Munic%c3%adpio'

r = requests.get(url)
data_json = r.json()
r.status_code

200

In [12]:
df = pd.DataFrame(data_json['items'])
df.head(5)

,id_pleito,tipo_interessado,interessado,cod_ibge,uf,num_pvl,status,num_processo,data_protocolo,tipo_operacao,finalidade,tipo_credor,credor,moeda,valor,pvl_assoc_divida,pvl_contradado_credor,data_status
0,5290,Município,Casimiro de Abreu,3301306,RJ,None,Deferido,17944.000078/2014-81,2014-11-28T02:00:00Z,Operação contratual interna,PAC 2 - Pró-Transporte - Pavimentação e Qualif...,Instituição Financeira Nacional,Caixa Econômica Federal,Real,15024627.28,1,0,12/12/2014
1,61009,Município,Rio das Flores,3304508,RJ,PVL02.007214/2022-19,Arquivado por decurso de prazo,17944.103071/2022-20,2022-08-12T18:48:50Z,Operação contratual interna,Infraestrutura,Instituição Financeira Nacional,Caixa Econômica Federal,Real,6000000.00,0,0,10/02/2023
2,66228,Município,São Pedro da Aldeia,3305208,RJ,PVL02.003177/2023-42,Deferido (PVL-IF),None,None,Operação contratual interna,Infraestrutura,Instituição Financeira Nacional,Banco do Brasil S/A,Real,5500000.00,0,0,12/09/2023
3,6363,Município,Rio Claro,3304409,RJ,None,Regularizado,17944.000328/2008-34,2011-06-01T03:00:00Z,Operação contratual interna,Regularização de Dívida - Energia Elétrica,Empresa Não Estatal,Light - Serviços de Eletricidade S/A,Real,6733230.99,0,0,22/07/2011
4,7907,Município,Mesquita,3302858,RJ,None,Deferido,17944.000675/2010-81,2010-09-06T03:00:00Z,Operação contratual interna,PNAFM,Instituição Financeira Nacional,Caixa Econômica Federal,Real,5106563.00,1,0,20/09/2010


In [13]:
# 2) Seu código aqui

df['status'].value_counts()
#resposta: Deferido, Arquivado e Regularizado com 105, 53 e 26 incidências

status
Deferido                                                 105
Arquivado                                                 53
Regularizado                                              26
Arquivado por decurso de prazo                            24
Deferido (PVL-IF)                                         24
Encaminhado à PGFN com manifestação técnica favorável     14
Arquivado a pedido                                         7
Em retificação pelo interessado                            3
Arquivado a pedido (PVL-IF)                                1
Arquivado pela STN                                         1
Em análise                                                 1
Name: count, dtype: int64

In [14]:
# 3) Seu código aqui

df['data_status'].astype(str)

df['ano_status'] = df['data_status'].str.slice(-4)
df['ano_status']

0      2014
1      2023
2      2023
3      2011
4      2010
       ... 
254    2014
255    2019
256    2019
257    2017
258    2007
Name: ano_status, Length: 259, dtype: object

In [15]:
# 4)
df['ano_status'].value_counts()

ano_status
2008    37
2007    29
2023    29
2014    20
2010    19
2019    16
2020    15
2013    15
2012    13
2022    11
2011    10
2009     9
2015     8
2017     5
2021     5
2002     4
2016     4
2018     4
2006     4
2003     1
2004     1
Name: count, dtype: int64

####  2) Melhorando a interação com o API
Observe dois URLs de consultas diferentes, por exemplo o URL utilizado em aula, e o URL feito no exercício anterior. Compare-os e observe as diferenças.

1. Faça uma função em Python que recebe como argumento o UF da consulta e o tipo de interessado (```'Estado'```ou ```Município```), e que devolve os dados da consulta no formato *DataFrame*.
2. Quantas solicitações para o Estado podem ser consultadas para Minas Gerais com *status* em 'Arquivado por decurso de prazo' estão registradas?
3. Qual é o município da Bahia com mais solicitações deferidas?
4. Salve um arquivo .csv com os dados de solicitações da Bahia, com interessado = 'Estado'

In [3]:
#1) Seu código aqui

def query_sadipem(uf = str, tipo_interessado = str):

    # Definindo variações da palavra município para que digitar sem acento também funcione, assim como uma opção padrão
    tipo_interessado_variations = ['município', 'estado']

    if tipo_interessado.lower() not in tipo_interessado_variations:
        tipo_interessado = 'município'
    #usando função quote do urrlib.parse para resolver de forma elegante a codificação do i com acento da palavra 'município' que pode causar problemas na URL
    tipo_interessado_encoded = quote(tipo_interessado)
    
    #definição de variáveis
    url = f'https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl?uf={uf.upper()}&tipo_interessado={tipo_interessado_encoded.capitalize()}'
    
    #puxando dados da url
    r = requests.get(url)
    
    #preparando json para manipulação
    data_json = r.json()
    
    #criação do DataFrame e exibição do mesmo
    if r.status_code == 200:
        df = pd.DataFrame(data_json['items'])
        print(url)
        return df
    else:
        print(f'Erro: Código {r.status_code}')

In [5]:
# 2) Seu código aqui

query_mg = query_sadipem('mg', 'estado')

query_mg

#resposta: 1

https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl?uf=MG&tipo_interessado=Estado


,id_pleito,tipo_interessado,interessado,cod_ibge,uf,num_pvl,status,num_processo,data_protocolo,tipo_operacao,finalidade,tipo_credor,credor,moeda,valor,pvl_assoc_divida,pvl_contradado_credor,data_status
0,4926,Estado,Minas Gerais,31,MG,None,Arquivado,10951.000367/1998-24,1998-05-12T03:00:00Z,Operação contratual externa (com garantia da U...,Infraestrutura,Instituição Financeira Internacional,Banco Internacional para Reconstrução e Desenv...,Dólar dos EUA,1.650000e+08,0,0,24/07/2002
1,7280,Estado,Minas Gerais,31,MG,None,Deferido,17944.000537/2010-01,2010-06-24T03:00:00Z,Operação contratual interna,Infraestrutura,Instituição Financeira Nacional,Banco Nacional de Desenvolvimento Econômico e ...,Real,3.488900e+07,1,0,25/06/2010
2,7435,Estado,Minas Gerais,31,MG,None,Encaminhado à PGFN com manifestação técnica fa...,17944.000576/2009-66,2009-05-26T03:00:00Z,Operação contratual externa (com garantia da U...,Fortalecimento Institucional,Instituição Financeira Internacional,Banco Interamericano de Desenvolvimento,Dólar dos EUA,4.000000e+07,1,0,10/06/2009
3,10338,Estado,Minas Gerais,31,MG,None,Encaminhado à PGFN com manifestação técnica fa...,17944.001175/2009-23,2009-09-22T03:00:00Z,Operação contratual interna (com garantia da U...,PEF - Programa Emergencial de Financiamento do...,Instituição Financeira Nacional,Banco do Brasil S/A,Real,1.781800e+08,1,0,08/10/2009
4,5828,Estado,Minas Gerais,31,MG,None,Encaminhado à PGFN com manifestação técnica fa...,17944.000208/2013-02,2013-07-16T03:00:00Z,Operação contratual interna (com garantia da U...,Infraestrutura,Instituição Financeira Nacional,Banco do Brasil S/A,Real,1.500000e+09,1,0,12/08/2013
5,13618,Estado,Minas Gerais,31,MG,None,Encaminhado à PGFN com manifestação técnica fa...,19405.000037/2006-15,2009-02-26T03:00:00Z,Operação contratual externa (com garantia da U...,Multissetorial,Instituição Financeira Internacional,Banco Interamericano de Desenvolvimento,Dólar dos EUA,1.000000e+07,1,0,12/03/2009
6,11054,Estado,Minas Gerais,31,MG,None,Encaminhado à PGFN com manifestação técnica fa...,17944.001346/2012-10,2012-10-29T02:00:00Z,Operação contratual interna (com garantia da U...,Proinveste,Instituição Financeira Nacional,Banco Nacional de Desenvolvimento Econômico e ...,Real,1.326390e+09,1,0,02/01/2013
7,20534,Estado,Minas Gerais,31,MG,00000.000000/2017-69,Arquivado por decurso de prazo,17944.001113/2015-60,2017-07-26T03:00:00Z,Operação contratual interna,PMAE,Instituição Financeira Nacional,Banco Nacional de Desenvolvimento Econômico e ...,Real,7.668000e+06,0,0,16/03/2018
8,11328,Estado,Minas Gerais,31,MG,None,Encaminhado à PGFN com manifestação técnica fa...,17944.001412/2012-51,2012-12-11T02:00:00Z,Operação contratual interna (com garantia da U...,Multissetorial,Instituição Financeira Nacional,Banco do Brasil S/A,Real,3.653733e+09,1,0,03/01/2013
9,22379,Estado,Minas Gerais,31,MG,00000.000000/2031-08,Arquivado pela STN,17944.000548/2016-78,2016-12-21T02:00:00Z,Operação contratual interna (com garantia da U...,Aditivo contratual,Instituição Financeira Nacional,Banco do Brasil S/A,Real,4.537330e+08,0,0,16/10/2017


In [55]:
# 3) Seu código aqui

query_ba = query_sadipem('ba', 'municipio')
query_ba['interessado'].value_counts()

#resposta: Camaçari

interessado
Camaçari                  19
Luís Eduardo Magalhães    18
Salvador                  18
Vitória da Conquista      15
Irecê                     14
                          ..
Serrolândia                1
Crisópolis                 1
Barro Alto                 1
Quijingue                  1
Santa Brígida              1
Name: count, Length: 297, dtype: int64

In [57]:
# 4) Seu código aqui

query_ba = query_sadipem('ba', 'estado')
query_ba.to_csv('./output/query_bahia_estado.csv', index= False, sep=';', decimal=',')